In [1]:
import spacy
import pickle
import random
import tqdm
import pandas as pd

C:\Users\aubin\AppData\Roaming\Python\Python38\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\aubin\AppData\Roaming\Python\Python38\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
C:\Users\aubin\AppData\Roaming\Python\Python38\site-packages\cupy\_environment.py:213: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


file = open("../stored_data/articles_cleaned.obj",'rb')
articles_cleaned = pickle.load(file)
file.close()

In [2]:
file = open("../stored_data/articles_with_tickers.obj",'rb')
articles_cleaned = pickle.load(file)
file.close()

In [3]:
nlp = spacy.load('../model/pipeline')

In [4]:
entity_type = [
    'EVENT',
    'FAC',
    'GPE',
    'LAW',
    'LOC',
    'NORP',
    'ORG',
    'PERSON',
    'PRODUCT',
    'WORK_OF_ART',
    'COMPANY',
    'COMMODITY'
]

In [5]:
id = list(articles_cleaned.keys())

random_id = random.sample(id, 1)
random_article = articles_cleaned[random_id[0]]

In [6]:
doc = nlp(random_article)

In [8]:
all_entities = dict((el,[]) for el in entity_type)
entity_with_fileid = {}

for file_id in tqdm.tqdm(id):
    entity_with_fileid[file_id] = []
    article = articles_cleaned[file_id]
    doc = nlp(article)
    for ent in doc.ents:
        if (ent.label_ != "DATE") and (ent.label_ != "CARDINAL") and (ent.label_ != "PERCENT") and (ent.label_ != "QUANTITY") and (ent.label_ != "ORDINAL") and (ent.label_ != "LANGUAGE") and (ent.label_ != "MONEY") and (ent.label_ != "TIME"):
            all_entities[ent.label_].append([ent.text, file_id])

100%|██████████| 8399/8399 [06:36<00:00, 21.19it/s]


In [9]:
all_entities.keys()

dict_keys(['EVENT', 'FAC', 'GPE', 'LAW', 'LOC', 'NORP', 'ORG', 'PERSON', 'PRODUCT', 'WORK_OF_ART', 'COMPANY', 'COMMODITY'])

In [10]:
# store in a dataframe with the following columns : 'id, name, entity, mentions'

In [19]:
count = 0
ent_to_article = {}

from collections import Counter
all_rows = []
for type in entity_type:
    selected_entities = all_entities[type]
    all_ents = []

    for sel in selected_entities:
        ent = sel[0]
        all_ents.append(ent)
        id = sel[1]

        if ent in list(ent_to_article.keys()):
            ent_to_article[ent].append(id)
        else:
            ent_to_article[ent] = []
            ent_to_article[ent].append(id)
    
    counted_entities = Counter(all_ents)
    for key, val in counted_entities.items():
        count += 1
        articles = ent_to_article[key]
        row = [count, key, type, val, articles]
        all_rows.append(row)


In [20]:
len(all_rows)

19762

In [23]:
entities = pd.DataFrame(all_rows, columns=['id', 'name', 'label', 'mentions', 'articles'])

In [25]:
entities.to_csv('../database/entities.csv')